## Ejercicios de pair programming 11 enero : ETL 3

Tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [27]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

import requests
import pandas as pd
import numpy as np
import ast

In [4]:
df = pd.read_csv("../datos/tiburon8.csv", index_col=0)

In [30]:
class Extraccion_union:
    def __init__(self, dicc_paises, df):
        self.dicc_paises = dicc_paises
        self.df = df

    def llamada_API (self):
        df_meteo = pd.DataFrame()  #Creamos un df vacio para ir añadiendo los resultados de las peticiones de informacion a la API por cada pais
        for k , v in self.dicc_paises.items():   #iteramos por cada key y value de nuestro diccionario
            lon = v[0]   #la longitud sera el valor 0 de nuestra lista de values
            lat= v[1]     #la latitud sera el valor 1 de nuestra lista de values
            url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product=meteo&output=json'     #llamamos a la url de la API
        
            response = requests.get(url=url)
            response.status_code
            response.reason

            meteo = response.json()
            meteo_data = meteo["dataseries"]  #necesitamos la info de la columna dataseries 
            meteo_df = pd.json_normalize(meteo_data)   #generamos un df 
            meteo_df["country"] = k.lower()   #creamos una columna llamada pais 
            df_meteo = pd.concat([df_meteo , meteo_df], axis= 0 , join= 'outer')  #concatenamos cada resultado de cada pais al df vacio
        return df_meteo

    def filtrado (self, list_paises):
        self.list_paises = list_paises

        df_filtrado = self.df[self.df['country'].isin(self.list_paises)]
        return df_filtrado


    def limpiar_clima(self, data):
        data = self.llamada_API()

        data['rh_profile']= data['rh_profile'].apply(ast.literal_eval)
        data['wind_profile']= data['wind_profile'].apply(ast.literal_eval)

        rh = data['rh_profile'].apply(pd.Series)

        for i in range(len(rh.columns)): 
            nombre = "rh_" + str(rh.iloc[0][i]['layer']) 
            valores = list(rh[i].apply(pd.Series)["rh"] )
            data.insert(i, nombre, valores) # Insertamos cada columna al df de clima


        wind = data['wind_profile'].apply(pd.Series)

        for i in range(len(wind.columns)): 
            nombre_dir = "wdir_" + str(wind.iloc[0][i]['layer']) 
            valores_dir = list(wind[i].apply(pd.Series)["direction"] )

            nombre_speed = "wspeed_" + str(wind.iloc[0][i]['layer']) 
            valores_speed = list(wind[i].apply(pd.Series)["speed"] )

            data.insert(i, nombre_dir, valores_dir) # Insertamos las columnas generadas en el df de clima
            data.insert(i, nombre_speed, valores_speed)


        data = data.groupby('country').mean()
        data = data.reset_index()
        return data

    def union_guardado(self, data, df_filtrado, nombre_guardado):
        data = self.limpiar_clima()
        df_filtrado = self.filtrado()

        self.nombre_guardado = nombre_guardado

        df_final = df_filtrado.merge(data, how='left',  on='country')

        df_final.to_csv(f'../datos/{self.nombre_guardado}')
        return df_final.head()

In [3]:
dicc_paises = {"USA" : [-100.445882, 39.7837304] , "Australia" : [134.755 , -24.7761086] , "South Africa" : [24.991639 , -28.8166236] , 
"New Zealand" : [172.8344077, -41.5000831], "Papua New Guinea": [144.2489081, -5.6816069]}  #Creamos el diccionario con los datos proporcionados

In [31]:
api = Extraccion_union(dicc_paises, df)

In [10]:
df_meteo = api.llamada_API()
df_meteo.head(2)


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 320, 'speed':...",13,15,5,1024,none,0,0,265,2,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 355, 'speed':...",12,15,5,1024,none,0,0,345,2,USA


In [32]:
lista_paises = ['usa', 'australia', 'new zealand', 'south africa', 'papua new guinea']

In [33]:
df_filtrado = api.filtrado(lista_paises)
df_filtrado.head(2)

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
2,2018.05.26.b,2018,Unprovoked,usa,Walking,Lower left leg bitten,May,N,M,Bull,15.0


In [34]:
clima = api.limpiar_clima()

/tmp/ipykernel_759/3465292018.py:60: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby('country').mean()


In [ ]:
df_meteo = api.llamada_API()

In [35]:
df_final = api.union_guardado('tiburon10.csv')

/tmp/ipykernel_759/3465292018.py:60: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby('country').mean()


TypeError: filtrado() missing 1 required positional argument: 'list_paises'